<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Objective-Function" data-toc-modified-id="Objective-Function-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Objective Function</a></span></li><li><span><a href="#Domain" data-toc-modified-id="Domain-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Domain</a></span></li><li><span><a href="#Optimization-Algoruthm" data-toc-modified-id="Optimization-Algoruthm-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Optimization Algoruthm</a></span></li><li><span><a href="#Results-History" data-toc-modified-id="Results-History-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Results History</a></span></li><li><span><a href="#Example-of-Sampling-from-the-Fomain" data-toc-modified-id="Example-of-Sampling-from-the-Fomain-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Example of Sampling from the Fomain</a></span></li></ul></div>

In [4]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [10]:
from sklearn.pipeline import make_pipeline, FeatureUnion, Pipeline
from src.feature_extraction import *
from src.util import *
from src.const import *
from sklearn.model_selection import KFold, train_test_split
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score

import csv
from hyperopt.pyll.stochastic import sample
from hyperopt import tpe, Trials, STATUS_OK, hp, fmin

from timeit import default_timer as timer
import ast



In [7]:
listing = pd.read_csv('../preprocessed_data/listings.csv')

In [8]:
price = pd.read_csv('../preprocessed_data/listings_price.csv', header=None, names=['price'])

In [11]:
for col in CATEGORY_COLUMNS:
    listing.loc[:, col] = listing.loc[:, col].astype('category')

In [12]:
train_features, test_features, train_labels, test_labels = train_test_split(listing, price, test_size=0.2, random_state=42)

print('Train shape: ', train_features.shape)
print('Test shape: ', test_features.shape)



Train shape:  (64613, 65)
Test shape:  (16154, 65)


In [13]:
N_FOLDS = 5
MAX_EVALS = 5

In [8]:
model = LGBMRegressor(random_state=50)

# Training set
train_set = lgb.Dataset(train_features, label = train_labels, categorical_feature=category_columns)
test_set = lgb.Dataset(test_features, label = test_labels, categorical_feature=category_columns)

In [9]:
hyperparameters = model.get_params()
print(hyperparameters)

{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.1, 'max_depth': -1, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 100, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': 50, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': True, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0}


In [10]:
# Using early stopping to determine number of estimators.
del hyperparameters['n_estimators']

In [11]:
# Perform cross validation with early stopping
cv_results = lgb.cv(hyperparameters, train_set,  num_boost_round = 10000, nfold = N_FOLDS, metrics = 'rmse', 
                    early_stopping_rounds = 100, verbose_eval = False, seed = 42, categorical_feature=category_columns,
                    stratified=False)

C:\Users\andre\Anaconda3\envs\DL\lib\site-packages\lightgbm\basic.py:741: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


In [12]:
best = cv_results['rmse-mean'][-1]

# Standard deviation of best score
best_std = cv_results['rmse-stdv'][-1]

print(f'The maximium RMSE in cross validation was {best:.5f} with std of {best_std:.5f}.')
print(f'The ideal number of iterations was {len(cv_results["rmse-mean"])}.')

The maximium RMSE in cross validation was 0.38774 with std of 0.00894.
The ideal number of iterations was 1117.


In [13]:
def evaluate(model, X_test, y_test):
    y_test_pred = model.predict(X_test)
    rmse_rf = (mean_squared_error(y_test, y_test_pred))**0.5
    r2score = r2_score(y_test, y_test_pred)
    adj_r2 = 1 - (1 - r2score) * (X_test.shape[0] - 1) / (X_test.shape[0] - X_test.shape[1] - 1)
    
    print(f'RMSE test: {rmse_rf}')
    print(f'R2 score test: {r2score}')
    print(f'Adjusted R2 score test: {adj_r2}')
    
    return rmse_rf, r2score

In [14]:
# Optimal number of esimators found in cv
model.n_estimators = len(cv_results['rmse-mean'])

# Train and make predicions with model
model.fit(train_features, train_labels)
preds = model.predict(test_features)
baseline_rmse = (mean_squared_error(test_labels, preds))**0.5
r2score = r2_score(test_labels, preds)

print(f'The baseline model has rmse {baseline_rmse:.5f} and r2score {r2score:.5f}')

The baseline model has rmse 0.38737 and r2score 0.76405


* Objective Function: takes in an input (hyperparameters) and returns a score to minimize or maximize (the cross validation score)
* Domain space: the range of input values (hyperparameters) to evaluate
* Optimization Algorithm: the method used to construct the surrogate function and choose the next values to evaluate
* Results: score, value pairs that the algorithm uses to build the surrogate function

### Objective Function

* take a set of hyperparameter values and return the cv score on the training data
* An objective function in Hyperopt must return either a single real value to minimize or a dictionary with a key "loss" with the score to minimize (and a key "status" indicating if the run was successful or not)

### Domain
* Space in hyperopt. In Hyperopt, and other Bayesian optimization frameworks, the domian is not a discrete grid but instead has probability distributions for each hyperparameter. For each hyperparameter, we will use the same limits as with the grid, but instead of being defined at each point, the domain represents probabilities for each hyperparameter.

### Optimization Algoruthm

* optimization algorithm is the method for constructing the surrogate function (probability model) and selecting the next set of hyperparameters to evaluate in the objective function. Either `random search` or `Tree Parzen Estimator`

### Results History
* objective function evaluations. 
    1. A `Trials` object that stores the dictionary returned from the objective function
    2. Adding a line to a csv file every iteration


In [58]:
def objective(hyperparameters):
    """Objective function for Gradient Boosting Machine Hyperparameter Optimization
    """
    # keep track of evals
    global ITERATION
    
    ITERATION += 1
    
    # Using early stopping to find number of trees trained
    if 'n_estimators' in hyperparameters:
        del hyperparameters['n_estimators']
    
    # Retrieve the subsample
    subsample = hyperparameters['boosting_type'].get('subsample', 1.0)
    
    # Extract the boosting type and subsample to top levl keys
    hyperparameters['boosting_type'] = hyperparameters['boosting_type']['boosting_type']
    hyperparameters['subsample'] = subsample
    
    # Make sure parameters that need to be integers are integers
    for parameter_name in ['num_leaves', 'subsample_for_bin', 'min_child_samples', 'max_depth']:
        hyperparameters[parameter_name] = int(hyperparameters[parameter_name])
    
    start = timer()
    
    cv_results = lgb.cv(hyperparameters, train_set, num_boost_round=10000, nfold=N_FOLDS,
                        early_stopping_rounds=100, metrics='rmse', 
                        categorical_feature=category_columns,
                        stratified=False, seed=50)
    
    run_time = timer() - start
    
    # Extract the best score
    best_score = cv_results['rmse-mean'][-1]
    
    # Boosting rounds that returned the highest cv score
    n_estimators = len(cv_results['rmse-mean'])
    
    # Add the number of estimators to the hyperparameters
    hyperparameters['n_estimators'] = n_estimators
    
    # write to the csv file ('a' means append)
    of_connection = open(OUT_FILE, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([best_score, hyperparameters, ITERATION, run_time, best_score])
    of_connection.close()
    
    # Dictionary with information for evaluation
    return {'loss': best_score, 'hyperparameters': hyperparameters, 'iteration': ITERATION,
            'train_time': run_time, 'status': STATUS_OK}
    

* The gbm cannot use the nested dictionary so we need to set the boosting_type and subsample as top level keys.
* Nested conditionals allow us to use a different set of hyperparameters depending on other hyperparameters. For example, we can explore different models with completely different sets of hyperparameters by using nested conditionals. The only requirement is that the first nested statement must be based on a choice hyperparameter (the choice could be the type of model).



In [62]:
space = {
    'boosting_type': hp.choice('boosting_type',
                               [{'boosting_type': 'gbdt', 'subsample': hp.uniform(
                                   'gbdt_subsample', 0.5, 1)},
                                {'boosting_type': 'dart', 'subsample': hp.uniform(
                                    'dart_subsample', 0.5, 1)},
                                {'boosting_type': 'goss', 'subsample': 1.0}]),
    'num_leaves': hp.quniform('num_leaves', 30, 255, 2),
    'max_depth': hp.quniform('max_depth', 3, 12, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.3)),
    'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0),
    'is_unbalance': hp.choice('is_unbalance', [True, False]),
    "device" : "gpu",
    "gpu_platform_id": 0,
    "gpu_device_id": 0,
}

### Example of Sampling from the Fomain

Sample from the domain (using the conditional logic) to see the result of each draw. Every time we run this code, the results will change. (Again notice that we need to assign the top level keys to the keywords understood by the GBM)

In [63]:
x = sample(space)
subsample = x['boosting_type'].get('subsample', 1.0)
x['boosting_type'] = x['boosting_type']['boosting_type']
x['subsample'] = subsample

x

{'boosting_type': 'gbdt',
 'colsample_bytree': 0.7333425547622549,
 'device': 'gpu',
 'gpu_device_id': 0,
 'gpu_platform_id': 0,
 'is_unbalance': False,
 'learning_rate': 0.2525576455996738,
 'max_depth': 11.0,
 'min_child_samples': 405.0,
 'num_leaves': 238.0,
 'reg_alpha': 0.17110537408961257,
 'reg_lambda': 0.7190327316157817,
 'subsample_for_bin': 220000.0,
 'subsample': 0.6693263909171743}

In [64]:
# Create a new file and open a connection
OUT_FILE = 'bayes_test.csv'
of_connection = open(OUT_FILE, 'w')
writer = csv.writer(of_connection)

ITERATION = 0

# Write column names
headers = ['loss', 'hyperparameters', 'iteration', 'runtime', 'score']
writer.writerow(headers)
of_connection.close()

# Test the objective function
results = objective(sample(space))
print('The cross validation loss = {:.5f}.'.format(results['loss']))
print('The optimal number of estimators was {}.'.format(results['hyperparameters']['n_estimators']))

The cross validation loss = 0.50813.
The optimal number of estimators was 18.


In [67]:
# Create the algorithm
tpe_algorithm = tpe.suggest

* `Trials` object will hold everything returned from the objective function in the `.results` attribute. . We can use this after the search is complete to inspect the results, but an easier method is to read in the csv file because that will already be in a dataframe.
* `fmin` takes the four parts defined above as well as the maximum number of iterations `max_evals`.

In [69]:
# record results
trials = Trials()

In [70]:
# Create a file and open a connection
OUT_FILE = 'bayes_test.csv'
of_connection = open(OUT_FILE, 'w')
writer = csv.writer(of_connection)

ITERATION = 0

# Write column names
headers = ['loss', 'hyperparameters', 'iteration', 'runtime', 'score']
writer.writerow(headers)
of_connection.close()


In [72]:
global ITERATION

ITERATION = 0

# Run optimization
best = fmin(fn = objective, space = space, algo = tpe.suggest, trials = trials,
            max_evals = MAX_EVALS)

best

100%|█████████████████████████████████████████████████| 5/5 [1:32:43<00:00, 1212.80s/it, best loss: 0.3824701385023954]


{'boosting_type': 0,
 'colsample_by_tree': 0.6249276122183668,
 'gbdt_subsample': 0.5020882484727693,
 'is_unbalance': 0,
 'learning_rate': 0.014146502081758888,
 'max_depth': 11.0,
 'min_child_samples': 95.0,
 'num_leaves': 192.0,
 'reg_alpha': 0.8564733637863978,
 'reg_lambda': 0.17706302999644274,
 'subsample_for_bin': 40000.0}

In [74]:
# Sort the trials with lowest loss first
trials_dict = sorted(trials.results, key=lambda x: x['loss'])
trials_dict[:1]

[{'loss': 0.3824701385023954,
  'hyperparameters': {'boosting_type': 'gbdt',
   'colsample_bytree': 0.6249276122183668,
   'device': 'gpu',
   'gpu_device_id': 0,
   'gpu_platform_id': 0,
   'is_unbalance': True,
   'learning_rate': 0.014146502081758888,
   'max_depth': 11,
   'min_child_samples': 95,
   'num_leaves': 192,
   'reg_alpha': 0.8564733637863978,
   'reg_lambda': 0.17706302999644274,
   'subsample_for_bin': 40000,
   'subsample': 0.5020882484727693,
   'n_estimators': 3083},
  'iteration': 1,
  'train_time': 516.6685468639989,
  'status': 'ok'}]

In [75]:
results = pd.read_csv(OUT_FILE)

In [80]:
def evaluate(results, name):
    """Evaluate model on test data using hyperparameter
       in results 
       
       Returns:
           hyp_df: dataframe of hyperparameters
    """
    
    new_results = results.copy()
    # string to dictionary
    new_results['hyperparameters'] = new_results['hyperparameters'].map(ast.literal_eval)
    
    # Sort with best values on top
    new_results = new_results.sort_values('score', ascending=True).reset_index(drop=True)
    
    # Print out cross validation high score
    print(f"The higest cross validation score from {name} was {new_results.loc[0, 'score']:.5f}"
          f" found on iteration {new_results.loc[0, 'iteration']}")
    
    # Use best hyperparameters to create a model
    hyperparameters = new_results.loc[0, 'hyperparameters']
    model = LGBMRegressor(**hyperparameters)
    
    # Train and make predictions
    model.fit(train_features, train_labels)
    preds = model.predict(test_features)
    rmse = (mean_squared_error(test_labels, preds))**0.5
    r2score = r2_score(y_test, y_test_pred)
    adj_r2 = 1 - (1 - r2score) * (test_features.shape[0] - 1) / (test_features.shape[0] - test_features.shape[1] - 1)
    print(f"RMSE from {name} on test data = {rmse:.5f}")
    print(f"R2 score from {name} on test data = {r2score:.5f}")
    print(f"Adjusted R2 score from {name} on test data = {adj_r2:.5f}")
    
    # Create dataframe of hyperparameters
    hyp_df = pd.DataFrame(columns=list(new_results.loc[0, 'hyperparameters'].keys()))
    
    # Iterate through each set of hyperparameters that were evaluted
    for i, hyp in enumerate(new_results['hyperparameters']):
          hyp_df = hyp_df.append(pd.DataFrame(hyp, index=[0]),
                                 ignore_index=True)
    
    # Put the iteration and score in the hyperparameter dataframe
    hyp_df['iteration'] = new_results['iteration']
    hyp_df['score'] = new_results['score']
    
    return hyp_df
    
    

In [81]:
bayes_results = evaluate(results, name = 'Bayesian')
bayes_results

The higest cross validation score from Bayesian was 0.38247 found on iteration 1
RMSE from Bayesian on test data = 0.38229


,boosting_type,colsample_bytree,device,gpu_device_id,gpu_platform_id,is_unbalance,learning_rate,max_depth,min_child_samples,num_leaves,reg_alpha,reg_lambda,subsample_for_bin,subsample,n_estimators,iteration,score
0,gbdt,0.624928,gpu,0,0,True,0.014147,11,95,192,0.856473,0.177063,40000,0.502088,3083,1,0.382470
1,gbdt,0.639034,gpu,0,0,True,0.016261,9,370,44,0.397152,0.827967,200000,0.844486,6001,5,0.387753
2,dart,0.901861,gpu,0,0,False,0.013189,10,180,154,0.504199,0.040143,160000,0.836686,10000,4,0.393371
3,goss,0.921651,gpu,0,0,True,0.019961,11,480,172,0.444405,0.453573,260000,1.000000,50,2,0.519208
4,goss,0.744739,gpu,0,0,True,0.228496,6,405,228,0.761490,0.075930,280000,1.000000,4,3,0.524774


In [82]:
import json

# save the trial results
with open('trials.json', 'w') as f:
    f.write(json.dumps(trials_dict))

In [83]:
MAX_EVALS = 200

# continue training
best = fmin(fn=objective, space=space, algo=tpe.suggest, trials=trials,
            max_evals=MAX_EVALS)

  1%|▌                                                  | 2/195 [00:31<57:29, 17.87s/it, best loss: 0.3824701385023954]

C:\Users\andre\Anaconda3\envs\DL\lib\site-packages\lightgbm\callback.py:189: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')



100%|████████████████████████████████████████████| 195/195 [40:40:28<00:00, 954.61s/it, best loss: 0.37978765352778115]


In [84]:
best

{'boosting_type': 0,
 'colsample_by_tree': 0.6887767729856713,
 'gbdt_subsample': 0.7418124791090016,
 'is_unbalance': 0,
 'learning_rate': 0.022964491720552586,
 'max_depth': 12.0,
 'min_child_samples': 20.0,
 'num_leaves': 172.0,
 'reg_alpha': 0.13480090927812335,
 'reg_lambda': 0.17660742377521615,
 'subsample_for_bin': 140000.0}

In [85]:
# Sort the trials with lowest loss first
trials_dict = sorted(trials.results, key=lambda x: x['loss'])
trials_dict[:1]

[{'loss': 0.37978765352778115,
  'hyperparameters': {'boosting_type': 'gbdt',
   'colsample_bytree': 0.6887767729856713,
   'device': 'gpu',
   'gpu_device_id': 0,
   'gpu_platform_id': 0,
   'is_unbalance': True,
   'learning_rate': 0.022964491720552586,
   'max_depth': 12,
   'min_child_samples': 20,
   'num_leaves': 172,
   'reg_alpha': 0.13480090927812335,
   'reg_lambda': 0.17660742377521615,
   'subsample_for_bin': 140000,
   'subsample': 0.7418124791090016,
   'n_estimators': 1932},
  'iteration': 180,
  'train_time': 439.9250337940175,
  'status': 'ok'}]

In [90]:
OUT_FILE = 'bayes_test.csv'
results = pd.read_csv(OUT_FILE)
bayes_results = evaluate(results, name = 'BayesianFinal')
bayes_results.head(5)

The higest cross validation score from BayesianFinal was 0.37979 found on iteration 180
RMSE from BayesianFinal on test data = 0.37856


,boosting_type,colsample_bytree,device,gpu_device_id,gpu_platform_id,is_unbalance,learning_rate,max_depth,min_child_samples,num_leaves,reg_alpha,reg_lambda,subsample_for_bin,subsample,n_estimators,iteration,score
0,gbdt,0.688777,gpu,0,0,True,0.022964,12,20,172,0.134801,0.176607,140000,0.741812,1932,180,0.379788
1,gbdt,0.733720,gpu,0,0,True,0.010406,12,25,126,0.095890,0.199557,120000,0.759393,5010,187,0.379794
2,gbdt,0.709935,gpu,0,0,False,0.014351,12,20,176,0.219238,0.628375,140000,0.573056,2945,97,0.379851
3,gbdt,0.663841,gpu,0,0,True,0.021814,12,20,136,0.308155,0.597981,160000,0.746613,1969,150,0.379930
4,gbdt,0.768120,gpu,0,0,True,0.010723,12,20,128,0.056426,0.096521,120000,0.784463,4835,189,0.380070


In [92]:
final_model = LGBMRegressor(random_state=50)

# Training set
train_set = lgb.Dataset(train_features, label = train_labels, categorical_feature=category_columns)
test_set = lgb.Dataset(test_features, label = test_labels, categorical_feature=category_columns)

find_model_params = trials_dict[0]['hyperparameters']

In [93]:
# Perform cross validation with early stopping
cv_results = lgb.cv(find_model_params, train_set,  num_boost_round = 10000, nfold = N_FOLDS, metrics = 'rmse', 
                    early_stopping_rounds = 100, verbose_eval = False, seed = 42, categorical_feature=category_columns,
                    stratified=False)

C:\Users\andre\Anaconda3\envs\DL\lib\site-packages\lightgbm\engine.py:430: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [ ]:
# Optimal number of esimators found in cv


# Train and make predicions with model
model.fit(train_features, train_labels)
preds = model.predict(test_features)
baseline_rmse = (mean_squared_error(test_labels, preds))**0.5
r2score = r2_score(test_labels, preds)

print(f'The baseline model has rmse {baseline_rmse:.5f} and r2score {r2score:.5f}')